<a href="https://colab.research.google.com/github/brnaoliveira/classificador-de-imagem/blob/master/classificador_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#ver se a GPU está funcionando
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
#Ativar o drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Arquitetura do linux e da máquina do colab
!uname -a
!cat /etc/lsb-release

Linux fdf8a373897a 4.14.79+ #1 SMP Wed Dec 19 21:19:13 PST 2018 x86_64 x86_64 x86_64 GNU/Linux
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04
DISTRIB_CODENAME=bionic
DISTRIB_DESCRIPTION="Ubuntu 18.04.2 LTS"


In [0]:
#Exemplo sem usar o keras puro


#Importando a biblioteca do keras
from tensorflow.keras import layers
from tensorflow.keras import Model
import os


# Codigo para montar o Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
base_dir = '/content/gdrive/My Drive/dataset'
train_dir = os.path.join(base_dir, 'training_set')
validation_dir = os.path.join(base_dir, 'test_set')


print(train_dir)
print(validation_dir)
# Diretório com as fotos de treinamento de gatos
train_cats_dir = os.path.join(train_dir, 'cats')

# Diretório com as fotos de treinamento de cachorros
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Diretório com as fotos de validação de gatos
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Diretório com as fotos de validação de cachorros
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

#Testar se o código ta dando certo
#train_cat_fnames = os.listdir(train_cats_dir)
#print(train_cat_fnames[:10])
#train_dog_fnames = os.listdir(train_dogs_dir)
#train_dog_fnames.sort()
#print(train_dog_fnames[:10])

#Quantidade de imagens
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

#Aperfeiçoamento da rede
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16

vgg_conv = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(224, 224, 3))


# Importar o gerador de dados de treino e teste
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importar NumPy
import numpy as np

# Guardar número de cães e gatos
nTrain = 8010
nVal = 2002

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
 
train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,2))
 
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')



validation_features = np.zeros(shape=(nVal, 7, 7, 512))
validation_labels = np.zeros(shape=(nVal,2))

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, 7 * 7 * 512))


#Criar a rede e mostrá-la
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from tensorflow.keras import optimizers

model = keras.Sequential()  # Observe que esta linha é diferente de Keras puro!

model.add(layers.Dense(512, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

history = model.fit(train_features,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(validation_features,validation_labels))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/dataset/training_set
/content/gdrive/My Drive/dataset/test_set
total training cat images: 4008
total training dog images: 4002
total validation cat images: 1001
total validation dog images: 1001
Found 8008 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


ValueError: ignored

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[pic_index-8:pic_index]]
next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 
  
  
  # Don't show axes (or gridlines)
  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [0]:
https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=Wpr8GxjOU8in


http://www.lapix.ufsc.br/ensino/visao-computacional/visao-computacionaldeep-learning/deep-learningintroducao-ao-novo-coneccionismo/deep-learningusando-nuvem-para-seus-trabalhos